In [33]:
import sklearn
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
import numpy as np
import pickle
import os

## Preparing Data
60% training, 20% validation, and 20% testing

In [68]:
# TODO: Move these two functions to a shared file for helper functions
def unPickleData(filename):
  with open(filename,'rb') as f: arr = pickle.load(f)
  return arr

def getDataPath(dirname,filename):
  return os.path.join(os.path.abspath(os.path.join(os.getcwd(), os.pardir, os.pardir)),"processed_data", dirname,filename)

x_data = unPickleData(getDataPath("comments","X_comments_sub.pkl"))
y_data = unPickleData(getDataPath("comments","Y_comments_sub.pkl"))

n = len(x_data)
x_data_training = x_data[:int(0.6*n)]
x_data_validation = x_data[int(0.6*n):int(0.8*n)]
x_data_testing = x_data[int(0.8*n):]

y_data_training = y_data[:int(0.6*n)]
y_data_validation = y_data[int(0.6*n):int(0.8*n)]
y_data_testing = y_data[int(0.8*n):]

## Training

In [69]:
model = LogisticRegression(multi_class='ovr',solver="liblinear") 
model.fit(x_data_training, y_data_training)

LogisticRegression(multi_class='ovr', solver='liblinear')

## Model Selection

In [70]:
grid = {"C":np.logspace(-3,3,100), "penalty":["l1","l2"]}

search = GridSearchCV(model,grid,cv=10)
search.fit(x_data_validation,y_data_validation)

print("Tuned hpyerparameters :(best parameters) ",search.best_params_)
print("Accuracy :",search.best_score_)

best_model = search.best_estimator_

Tuned hpyerparameters :(best parameters)  {'C': 17.47528400007683, 'penalty': 'l2'}
Accuracy : 0.8732804232804232


## Testing

### Excluding the validation set

In [71]:
y_pred = best_model.predict(x_data_testing)

print(sklearn.metrics.classification_report(y_data_testing,y_pred))
print("Accuracy: ", sklearn.metrics.accuracy_score(y_data_testing, y_pred))

              precision    recall  f1-score   support

           0       0.88      0.98      0.92       229
           1       0.76      0.33      0.46        48

    accuracy                           0.87       277
   macro avg       0.82      0.66      0.69       277
weighted avg       0.86      0.87      0.84       277

Accuracy:  0.8664259927797834


### Including the validation set
Because of the lack of representation of the class "bot", especially in the testing data, the model reflects poor results when it comes to the recall and f-score.

In [72]:
y_pred = best_model.predict(np.concatenate((x_data_testing,x_data_validation))
)
print(sklearn.metrics.classification_report(np.concatenate((y_data_testing,y_data_validation)),y_pred))
print("Accuracy: ", sklearn.metrics.accuracy_score(np.concatenate((y_data_testing,y_data_validation)), y_pred))

              precision    recall  f1-score   support

           0       0.91      0.98      0.95       459
           1       0.88      0.52      0.65        94

    accuracy                           0.91       553
   macro avg       0.89      0.75      0.80       553
weighted avg       0.90      0.91      0.90       553

Accuracy:  0.9059674502712477
